In [ ]:
!pip install langchain
!pip install langchain_community
!pip install huggingface_hub
!pip install langchain_core
!pip install langchain_groq
!pip install langchain groq
!pip install faiss-cpu
!pip install InstructorEmbedding
!pip install sentence-transformers==2.2.2
!pip install torch
!pip install transformers

In [ ]:
import os
import json
from langchain_community.embeddings import HuggingFaceInstructEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.chains import ConversationalRetrievalChain
from langchain_groq import ChatGroq
from langchain.memory import ConversationBufferMemory
from langchain.schema import Document

In [ ]:
folder_path = '/content/drive/MyDrive/chunks'

'''def load_chunks(folder_path):

  for file in os.listdir(folder_path):
    text_chunks = []
    #extracting chunks content
    with open(os.path.join(folder_path, file), 'r') as f:
      chunk_content = json.load(f)
      texts = [item['chunk'] for item in chunk_content]
      text_chunks = ' '.join(texts)
      text_chunks.append(Document(page_content=text_chunks))

  return text_chunks'''
def load_chunks(folder_path):
  all_text_chunks = []
  print("Files found:", os.listdir(folder_path)) # Check if files are found
  for file in os.listdir(folder_path):
    text_chunks = []
    #extracting chunks content
    with open(os.path.join(folder_path, file), 'r') as f:
      chunk_content = json.load(f)
      print("Chunks in file", file, ":", len(chunk_content)) # Check number of chunks
      texts = [item['chunk'] for item in chunk_content]
      combined_text = ' '.join(texts)
      text_chunks.append(Document(page_content=combined_text))
    all_text_chunks.extend(text_chunks)

  print("Total chunks loaded:", len(all_text_chunks)) # Check total chunks
  return all_text_chunks



In [ ]:
#vector database
def get_vectorstore(text_chunks):
  if not text_chunks:
    print("No text chunks to create embeddings")
    return None

  embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-xl")
  vectorstore = FAISS.from_texts(texts=[t.page_content for t in text_chunks], embedding=embeddings)
  return vectorstore

In [ ]:
# Creating a conversation chain to store the context for follow-up questions
def get_conversation_chain(vectorstore, groq_api_key):
    llm = ChatGroq(groq_api_key=groq_api_key, model_name="llama-3.1-70b-versatile")
    memory = ConversationBufferMemory(
        memory_key='chat_history', return_messages=True)
    conversation_chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        retriever=vectorstore.as_retriever(),
        memory=memory
    )
    return conversation_chain

In [ ]:
# Handling the user input
def handle_userinput(conversation_chain, user_question):
    response = conversation_chain({'question': user_question})
    chat_history = response['chat_history']

    for i, message in enumerate(chat_history):
        if i % 2 == 0:
            print(f"User: {message.content}")
        else:
            print(f"Bot: {message.content}")

In [ ]:
#driver function
def main():
  #your groq api key
  groq_api_key = "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"
  #folder path to get chunks
  folder_path = '/content/drive/MyDrive/chunks'

  text_chunks = load_chunks(folder_path)
  vectorstore = get_vectorstore(text_chunks)
  if vectorstore:
        # Create conversation chain
        conversation_chain = get_conversation_chain(vectorstore, groq_api_key)

        while True:
            user_question = input("Ask something (type 'exit' to quit): ")
            if user_question.lower() in ['exit' , 'quit']:
                break
            handle_userinput(conversation_chain, user_question)

if __name__ == "__main__":
    main()

Files found: ['Python Pandas Tutorial (Part 5): Updating Rows and Columns - Modifying Data Within DataFrames.json', 'Matplotlib Tutorial (Part 9): Plotting Live Data in Real-Time.json']
Chunks in file Python Pandas Tutorial (Part 5): Updating Rows and Columns - Modifying Data Within DataFrames.json : 8
Chunks in file Matplotlib Tutorial (Part 9): Plotting Live Data in Real-Time.json : 5
Total chunks loaded: 2


/usr/local/lib/python3.10/dist-packages/InstructorEmbedding/instructor.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange


load INSTRUCTOR_Transformer
max_seq_length  512
Ask something (type 'exit' to quit): hey


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  warn_deprecated(


User: hey
Bot: How can I help you today?
Ask something (type 'exit' to quit): what is the content provide to you is about
User: hey
Bot: How can I help you today?
User: what is the content provide to you is about
Bot: I have been provided with two transcripts of videos. The first transcript appears to be about creating real-time plots using matplotlib in Python, and the second transcript is about updating existing rows and columns in pandas data frames.
Ask something (type 'exit' to quit): exit
